In [1]:
import tkinter
from tkinter import *

import PIL
import PIL.Image 
from PIL import ImageTk

import nltk
from nltk.stem import WordNetLemmatizer

import pickle

import numpy as np

from keras.models import load_model

import json

import random

from keras.preprocessing.image import load_img, img_to_array

Using TensorFlow backend.


In [2]:
# load and prepare the image
def load_image(filename):
    # load the image
    img = load_img(filename, target_size=(200, 200))
    # convert to array
    img = img_to_array(img)
    # reshape into a single sample with 3 channels
    img = img.reshape(1, 200, 200, 3)
    return img

In [3]:
def chatbot_response_images(msg):
    
    img = load_image(msg)
    modelCNN = load_model('final_model.h5')
    result = modelCNN.predict(img)
    
    if result[0] == 0.:
        result = "cat"
    else:
        result = "dog"
    
    return result

In [4]:
lemmatizer = WordNetLemmatizer()

model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [5]:
def chatbot_response_text(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    
    return res

In [6]:
#Creating GUI with tkinter
import tkinter
from tkinter import *

def send():
    
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)


    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.config(foreground="#442265", font=("Verdana", 12 )) 
        
        if ".jpg" in msg or ".jpeg" in msg or ".png" in msg:
            #shows what the user entered into the entry box
            ChatLog.insert(END, "YOU:\n")
            img = PIL.Image.open(msg)
            img = img.resize((100,100), PIL.Image.ANTIALIAS)
            photo = ImageTk.PhotoImage(img)
            label = Label(image=photo)
            label.image = photo
            ChatLog.image_create(END, image=photo)
            ChatLog.insert(END, "\n\n")
            
            #shows what the computer response is
            res = chatbot_response_images(msg)
            ChatLog.insert(END, "BOT:\nI predict that the subject of the picture is a " + res + "\n\n")

            
        else:
            #shows what the user entered into the entry box
            ChatLog.insert(END, "YOU:\n" + msg + "\n\n")
            
            #shows what the computer response is 
            res = chatbot_response_text(msg)
            ChatLog.insert(END, "Bot:\n" + res + '\n\n')
        
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

#function is called when user presses Enter button
def enterSend(event):
    send()    
    
    
base = Tk()
base.title("Hello Bot")
base.geometry("460x550")
base.resizable(width=TRUE, height=TRUE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", font="Arial",)
ChatLog.config(state=DISABLED)

#Scrollbar
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command=send)


#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#f5f5f5", width="29", height="5", font="Arial")
EntryBox.bind("<Return>", enterSend)

#for grid geometry
base.columnconfigure(1, weight=1)
base.rowconfigure(3, weight=1)


lbl = Label(base, text="Say 'Hi' to Hello Bot!")
lbl.grid(sticky=W)

ChatLog.grid(row=1, column=0, rowspan=4, columnspan=4, sticky=E+W+S+N)
scrollbar.grid(row=1, column=4, rowspan=4, sticky=E+W+S+N)
EntryBox.grid(row=5, column=1, columnspan=4, sticky=E+W+S+N)
SendButton.grid(row=5, column=0, columnspan=1, sticky=E+W+S+N)


ChatLog.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=ChatLog.yview)


base.mainloop()